<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo I: Regresión Logística para Análisis de Sentimientos

## Importar librerías y funciones

In [1]:
import nltk
from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

from utilss import Utilities as prep



### Prepara los datos
* `twitter_samples` contiene subconjuntos de 5,000 tweets positivos, 5,000 tweets negativos y el conjunto completo de 10,000 tweets.
     * Si utiliza los tres conjuntos de datos, introduciríamos duplicados de los tweets positivos y negativos.
     * Seleccionará solo los cinco mil tweets positivos y los cinco mil tweets negativos.

In [2]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')




* Train test split: 20% para test, y 80% para train.


In [3]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
print(len(test_pos))
print(len(train_pos))



1000
4000


In [4]:
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]
print(len(test_neg))
print(len(train_neg))



1000
4000


In [5]:
train_x = train_pos + train_neg
test_x = test_pos + test_neg



* Creear una matriz de etiquetas positivas y negativas.

In [6]:
train_y = np.append(np.ones((len(train_pos),1)), np.zeros((len(train_neg),1)), axis=0)
test_y = np.append(np.ones((len(test_pos),1)), np.zeros((len(test_neg),1)), axis=0)


In [7]:
print(len(train_y))

8000


In [8]:
print(len(test_y))

2000


In [9]:
# Crear el diccionario de frecuencias
prep_obj = prep()


freqs = prep_obj.build_freqs(train_x, train_y)
len(freqs)

This is the Utilities Constructor


11340

### Procesamiento del tweet

La función dada `process_tweet ()` tokeniza el tweet en palabras individuales, elimina las palabras vacías y aplica la derivación.

In [10]:
print(train_x[1234])
prep_obj.process_tweet(train_x[1234])



@leomanaids_ look here u little shit :-) I fell asleep &amp; then woke up &amp; thought u were asleep :-) PLUS this was earlier :-) so fuk u dude


['look',
 'u',
 'littl',
 'shit',
 ':-)',
 'fell',
 'asleep',
 'woke',
 'thought',
 'u',
 'asleep',
 ':-)',
 'plu',
 'earlier',
 ':-)',
 'fuk',
 'u',
 'dude']

##  Extrayendo las características

* Dada una lista de tweets, extraiga las características y guárdelas en una matriz. Extraerás dos características.
     * La primera característica es la cantidad de palabras positivas en un tweet.
     * La segunda característica es la cantidad de palabras negativas en un tweet.
* Luego entrene su clasificador de regresión logística en estas características.
* Pruebe el clasificador en un conjunto de validación. 

### Implementación de la función de extract_features.
* Esta función admite un solo tweet.
* Procesaremos el tweet usando la función `process_tweet()` importada y la guardaremos en la lista de palabras del tweet.
* Recorreremos cada palabra en la lista de palabras procesadas
     * Para cada palabra, consultaremos el diccionario `freqs` para el recuento cuando esa palabra tiene una etiqueta positiva '1'. (con clave (palabra, 1.0)
     * Hacemos lo mismo con el recuento para cuando la palabra esté asociada con la etiqueta negativa '0'. (con la clave (palabra, 0.0).)


In [11]:
def extract_features(tweet, freqs):
    
    word_l = prep_obj.process_tweet(tweet)
    
    x= np.zeros((1,3))
    
    x[0,0]=1
    
    # y = b_0 + b_1 * x_1 + b_2 * x_2
    # X= [1 ;x_1; x2]
    # B=[b_0; b1; b2]
    
    # y = B*X
    
    for word in word_l:
        
        # incrementar el conteo de palabras para la etiqueta positiva (1)
        x[0,1] += freqs.get((word,1),0)
        
        # incrementar el conteo de palabras para la etiqueta negativa (0)
        x[0,2] += freqs.get((word,0),0)
        
    return x 
    


In [12]:
tmpl = extract_features(train_x[1234], freqs)

In [13]:
tmpl.shape

(1, 3)

In [14]:
print(tmpl)

[[1.000e+00 2.239e+03 6.800e+02]]


# Implementación de la Regresión Logística


### Función sigmoide
Aprenderá a utilizar la regresión logística para la clasificación de texto.
* La función sigmoidea se define como:

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Asigna la entrada 'z' a un valor que varía entre 0 y 1, por lo que puede tratarse como una probabilidad.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='Figures/sigmoid_plot.jpg' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> </div>

In [15]:
def sigmoid(z):
    h = 1/(1+np.exp(-z))
    return h



In [16]:
sigmoid(0)

0.5

In [17]:
sigmoid(4.9)

0.9926084586557181

In [18]:
sigmoid(-1)

0.2689414213699951

### Logistic regression: regression y función sigmoide

La regresión logística toma una regresión lineal regular y aplica un sigmoide a la salida de la regresión lineal.

Regresion:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Tenga en cuenta que los valores $ \theta $ son "pesos". Si realizó la especialización en aprendizaje profundo, nos referimos a los pesos con el vector `w`. En este curso, usamos una variable diferente $ \theta $ para referirnos a los pesos.

Regresión logística
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Nos referiremos a 'z' como los 'logits'.

### Función de costo y gradiente

La función de costo utilizada para la regresión logística es el promedio de la pérdida de registro en todos los ejemplos de entrenamiento:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ es la cantidad de ejemplos de entrenamiento
* $y^{(i)}$ es la etiqueta real del i-ésimo dato de entrenamiento.
* $h(z(\theta)^{(i)})$ es la predicción del modelo para el i-ésimo ejemplo de entrenamiento.

La función de pérdida para un solo ejemplo de entrenamiento es
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Todos los valores de $ h $ están entre 0 y 1, por lo que los registros serán negativos. Esa es la razón del factor -1 aplicado a la suma de los dos términos de pérdida.
* Tenga en cuenta que cuando el modelo predice 1 ($ h (z (\theta)) = 1 $) y la etiqueta $ y $ también es 1, la pérdida para ese ejemplo de entrenamiento es 0.
* De manera similar, cuando el modelo predice 0 ($ h (z (\theta)) = 0 $) y la etiqueta real también es 0, la pérdida para ese ejemplo de entrenamiento es 0.
* Sin embargo, cuando la predicción del modelo es cercana a 1 ($ h (z (\theta)) = 0.9999 $) y la etiqueta es 0, el segundo término de la pérdida logarítmica se convierte en un gran número negativo, que luego se multiplica por el factor general de -1 para convertirlo en un valor de pérdida positivo. $ -1 \times (1 - 0) \times log (1 - 0.9999) \approx 9.2 $ Cuanto más se acerque la predicción del modelo a 1, mayor será la pérdida.

* Del mismo modo, si el modelo predice cerca de 0 ($ h (z) = 0.0001 $) pero la etiqueta real es 1, el primer término en la función de pérdida se convierte en un número grande: $ -1 \times log (0.0001) \approx 9.2 $. Cuanto más cercana sea la predicción a cero, mayor será la pérdida.

#### Actualizar los pesos

Para actualizar su vector de peso $ \theta $, aplicará el descenso de gradiente para mejorar iterativamente las predicciones de su modelo.
El gradiente de la función de costo $ J $ con respecto a uno de los pesos $ \theta_j $ es:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$
* 'i' es el índice de todos los ejemplos de formación "m".
* 'j' es el índice del peso $ \theta_j $, entonces $ x_j $ es la característica asociada con el peso $ \theta_j $

* Para actualizar el peso $ \theta_j $, lo ajustamos restando una fracción del gradiente determinado por $ \alpha $:
$$ \theta_j = \theta_j - \alpha \times \nabla_{\theta_j} J (\theta) $$
* La tasa de aprendizaje $ \alpha $ es un valor que elegimos para controlar qué tan grande será una sola actualización.


## Implementación de la función Gradiente Descendente
* El número de iteraciones `num_iters` es el número de veces que utilizará todo el conjunto de entrenamiento.
* Para cada iteración, calculará la función de costo usando todos los ejemplos de entrenamiento (hay ejemplos de entrenamiento `m`), y para todas las funciones.
* En lugar de actualizar un solo peso $ \theta_i $ a la vez, podemos actualizar todos los pesos en el vector de columna:  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* $ \mathbf {\theta} $ tiene dimensiones (n + 1, 1), donde 'n' es el número de características, y hay un elemento más para el término de sesgo $ \theta_0 $ (tenga en cuenta que el valor de característica correspondiente $ \mathbf {x_0} $ es 1).
* Los 'logits', 'z', se calculan multiplicando la matriz de características 'x' con el vector de peso 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1) 
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* La predicción 'h' se calcula aplicando el sigmoide a cada elemento en 'z': $ h (z) = sigmoid (z) $, y tiene dimensiones (m, 1).
* La función de costo $ J $ se calcula tomando el producto escalar de los vectores 'y' y 'log (h)'. Dado que tanto 'y' como 'h' son vectores de columna (m, 1), transponga el vector a la izquierda, de modo que la multiplicación de matrices de un vector de fila con un vector de columna realice el producto escalar.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* La actualización de theta también está vectorizada. Debido a que las dimensiones de $ \mathbf {x} $ son (m, n + 1), y tanto $ \mathbf {h} $ como $ \mathbf {y} $ son (m, 1), necesitamos transponer $ \mathbf {x} $ y colóquelo a la izquierda para realizar la multiplicación de matrices, que luego da la respuesta (n + 1, 1) que necesitamos:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [19]:
def gradientDescent(x, y, theta, alpha, num_iters):
    
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        z = np.dot(x,theta)
        
        h = sigmoid(z)
        
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(), np.log(1-h)))
        
        theta = theta -(alpha/m)*np.dot(x.transpose(),(h-y))
        
        
    J = float(J)
    
    return J, theta     
        
        
    

In [20]:
len(train_x)

8000

In [21]:
X = np.zeros((len(train_x),3))


In [22]:
X

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [23]:
tmpl


array([[1.000e+00, 2.239e+03, 6.800e+02]])

In [24]:
for i in range(len(train_x)):
    X[i,:] = extract_features(train_x[i], freqs)



In [25]:
X

array([[1.000e+00, 3.020e+03, 6.100e+01],
       [1.000e+00, 3.573e+03, 4.440e+02],
       [1.000e+00, 3.005e+03, 1.150e+02],
       ...,
       [1.000e+00, 1.440e+02, 7.830e+02],
       [1.000e+00, 2.050e+02, 3.890e+03],
       [1.000e+00, 1.890e+02, 3.974e+03]])

In [26]:
Y = train_y

In [27]:
theta_0 = np.zeros((3,1))
alpha = 1e-9
niter= 1500

J, theta = gradientDescent(X, Y, theta_0, alpha, niter)


In [28]:
J

0.24216476596262507

In [30]:
theta

array([[ 7.25227840e-08],
       [ 5.23899773e-04],
       [-5.55170736e-04]])

In [31]:
#thetha[0] + theta[1]*x1 + theta[2]*x2

tweet_ejemplo="great week"


In [32]:
def predict_tweet(tweet, freqs, theta):
    
    x = extract_features(tweet, freqs)
    
    y_pred = sigmoid(np.dot(x,theta))
    
    return y_pred
    
    

In [34]:
predict_tweet(tweet_ejemplo,freqs, theta)

array([[0.51836744]])

In [41]:
tweet_list = ['happy','happy weekend', 'buena inundacion', 'nice work asshole', 'bad', 'this movie was very bad']

In [42]:
for tweet in tweet_list:
    print('%s -> %f' % (tweet, predict_tweet(tweet,freqs, theta)))

happy -> 0.518580
happy weekend -> 0.523646
buena inundacion -> 0.500000
nice work asshole -> 0.504301
bad -> 0.494339
this movie was very bad -> 0.494206
